<a href="https://colab.research.google.com/github/MINDS-edu/The-5th-POSTECH-Youth-Mathematical-Artificial-Intelligence-Academy/blob/master/PYMAIA5_day1_선형회귀.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



[![](https://www.gstatic.com/images/branding/product/2x/sheets_2020q4_48dp.png)데이터 수집용](https://docs.google.com/spreadsheets/d/15IIOeoywneGBK2IiAdYUgn8RBUWCPZawcfoBW1AB6OU/edit?usp=sharing)


#1 데이터 전처리 및 탐색


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import io
import pandas as pd


## 데이터 수집

In [ ]:
df = pd.read_csv(io.StringIO('''
ID,키,발 사이즈,"성별(남,여)"
차은우,167,250,여
홍지후,162,245,여
"ㅡ""ㅡ",164,255,여
  ll,168,240,여
장윤아,162,240,여
수혀니,165,230,여
미미민지,158,230,여
김노겸,159,240,여
애기공주,198,320,여
집가고싶다고,167,230,여
ㅇ,186,280,남
ㅇ[]ㅇ,161,250,여
송재민,184,290,남
정세헌,174,270,남
권하율,170,270,남
박지원,181,300,남
류민석,151,220,여
악,156,245,여
이세한,176,265,남
ㅇㅅㅇ,162,245,남
이연지,165,245,여
어니ㅏㅇ머,162,250,여
윤하준,175,265,상남자
이름,176,270,남
'''))
df

## 데이터 전처리

한국어 -> 영어

In [ ]:
df.columns = ['ID','height','shoe size','sex']

결측치 처리

In [ ]:
gender_map = {
    '남': 'male',
    '상남자': 'male',  # '상남자'도 'male'로 매핑
    '여': 'Female'
}
df['sex'] = df['sex'].map(gender_map)
df

In [ ]:
df = df.iloc[:,1:]

불필요한 데이터 제거

In [ ]:
df

## 데이터 시각화

시각화1: 산점도 플롯

In [ ]:
xs = df['height'].values
ys = df['shoe size'].values
fig,ax = plt.subplots()
ax.plot(xs,ys,'o')
plt.ylabel('Shoe Size')
plt.xlabel('Height')


시각화2 = 시각화1 + 성별 정보  
->좋은 시각화는 데이터를 더 명료하게 보여준다.

In [ ]:
import seaborn as sns
sns.scatterplot(data=df, x='height', y='shoe size',hue='sex')

이상치 제거

In [ ]:
is_outlier = df['height'].values>190
df = df[~is_outlier]
sns.scatterplot(data=df, x='height', y='shoe size',hue='sex')

# 2 키로 발 사이즈를 예측하는 선형회귀 모델 만들기
학생들의 키와 발 사이즈 사이의 관계를 보았을 때, 키가 180cm인 선생님의 발 사이즈는 몇일까요? 산점도에 따르면 280~290mm 사이일 것이라고 예상되죠? 만약 선생님뿐만 아니라 100명이 넘는 사람의 발 사이즈를 예상해야 한다고 하면 이렇게 눈으로 어림 잡아서 말하기가 굉장히 힘들 것이에요. 우리는 "키가 $x$ cm면 발 사이즈는 $y$ mm이다"라고 딱 말해줄 수 있는 모델이 필요합니다. 그래서 우리는 키로 발 사이즈를 예측하는 모델을 만들어 볼 것입니다. 특히, 우리는 1차 함수를 사용하여 과학 점수를 예측할 것입니다. 1차 함수는 다음과 같이 적어줄 수 있습니다.
$$f(x) = ax + b$$

<br>

이때, $a$는 기울기, $b$는 y절편이죠? $a$와 $b$에 어떤 값이 오느냐에 따라서 직선의 모양이 달라지게 됩니다. 그리고 $x$에 어떤 값을 넣느냐에 따라서 $f(x)$ 값이 달라집니다. 우리의 경우 $x$는 키, $f(x)$의 경우 발 사이즈라고 생각할 수 있습니다.

$$\text{발사이즈} = a \times \text{키} + b$$

<br>

모델이 입력 받는 변수를 **독립 변수** 그리고 모델이 예측하고자 하는 변수를 **종속 변수**라고 부릅니다. 지금은 하나의 독립 변수를 이용해서 하나의 종속 변수를 예측하는 모델을 만들고 있는 것입니다. 우리의 목표는 데이터를 가장 잘 설명할 수 있는 기울기와 y절편을 갖는 모델을 찾아내는 것입니다. $a$와 $b$와 같이 모델의 모양을 결정해주는 변수들을 **매개변수** 또는 **파라미터**라고 부릅니다. 이와 같이 독립 변수에 매개변수 곱한 다음 모두 더해주는 모델을 **선형 모델**이라고 부릅니다.

In [ ]:
# @title
xs = df['height'].values
ys = df['shoe size'].values

plt.figure()
plt.plot(xs, ys, 'o')
plt.xlabel('Height')
plt.ylabel('Shoe Size')
plt.show()

## 파라미터 튜닝: a, b를 조절해서 적당한 직선 찾기
아래의 셀에 기울기 $a$가 1이고 y절편 $b$가 0일 때 일차함수를 그려주는 코드가 있습니다. 이 일차함수는 데이터를 잘 설명하고 있나요? 그럼, 아래의 셀에서 `a` 변수와 `b` 값을 바꿔가면서 데이터를 가장 잘 설명할 수 있는 직선을 찾아보세요.

In [ ]:
a = 1 # @param {type:"raw"}
b = 0 # @param {type:"raw"}

plt.figure()
plt.plot(xs, ys, 'o', label='Actual Data')
plt.ylabel('Shoe Size')
plt.xlabel('Height')
xx = np.arange(150, 200)
plt.plot(xx, a * xx + b, label='Predict')
plt.legend()
plt.title('Linear Regression')
plt.show()

여러분들은 어떤 `a`와 `b` 값을 사용하셨나요? 여러분들이 만든 모델에 따르면 키가 180cm인 선생님의 발사이즈는 어떻게 되나요?

In [ ]:
print("내 모델이 예측한 선생님 발 사이즈:", a * 180 + b)

다른 친구들은 어떻게 했을까?




<br>

## 비용 함수: 좋은 직선을 수학적으로 정의하기

결과를 공유해준 학생들 중 어떤 학생의 직선이 가장 좋은 직선일까요? 다시 말해 어떤 파라미터가 좋은 파라미터이고 어떤 파라미터가 나쁜 파라미터일까요? 파라미터의 좋고 나쁨을 따지기 위해 우리는 비용 함수 (loss function)라는 것을 정의합니다.

- 파라미터가 정해지면 모델 (우리의 경우 일차함수)의 모양이 결정됩니다.
- 모델의 모양이 결정되면, 각 데이터마다 모델이 예측한 발 사이즈 $f(x)$와 실제 학생의 발 사이즈의 차이 (**오차**)를 계산할 수 있게 됩니다.
  - $i$번째 학생의 키를 $x_i$, 발 사이즈를 $y_i$라고 할 때,
  - $i$번째 학생에 대한 모델의 오차
  $$\begin{matrix}r_i&=&(y_i - f(x_i))^2 \\ & = & \left( y_i - (ax_i+b) \right)^2 \end{matrix}$$

   - 발 사이즈를 잘 예측하는 모델일수록 예측한 발 사이즈와 실제 발 사이즈 사이의 오차가 작을 것입니다.
- 우리는 모든 데이터에서 발생한 오차의 제곱의 평균 (또는 합)을 가장 작게 만들어주는 매개변수를 찾아서 사용하게 됩니다.
  - 아래 비용 함수를 Mean Squared Error (MSE)라고 부름

$$
\begin{matrix}
S(a, b) & = &\displaystyle \frac{1}{21} \sum\limits_{i=1}^{21}\left( y_i - f(x_i) \right)^2 \\
& = &\displaystyle \frac{1}{21} \sum\limits_{i=1}^{21}\left( y_i - (ax_i+b)\right)^2
\end{matrix}
$$


In [ ]:
a = 1# @param {type:"number"}
b = 0# @param {type:"number"}
fcn_estimate = lambda x: a * x + b
xs = df['height'].values
ys = df['shoe size'].values

def sort_by_x(x_unsorted, y_unsorted):
    """
    Sorts two lists/arrays based on the values in the first list/array.

    Args:
        x_unsorted: The list or array to sort by.
        y_unsorted: The list or array to sort according to the order of x_unsorted.

    Returns:
        A tuple containing the sorted x_unsorted and y_unsorted.
    """
    # Combine x and y into pairs, sort by x, then separate
    sorted_pairs = sorted(zip(x_unsorted, y_unsorted))
    x_sorted, y_sorted = zip(*sorted_pairs)

    return np.array(x_sorted), np.array(y_sorted)


def visualize_MSE(fcn_estimate, xs, ys, l_compare=100):
  l = l_compare
  xs, ys = sort_by_x(xs, ys)
  fcn = fcn_estimate

  fig, axs = plt.subplots(1, 2, layout='constrained',figsize=(10, 5))
  plt.sca(axs[0])
  plt.plot(xs, ys, 'o', zorder=3, label='Actual Data')
  x_smooth = np.linspace(xs.min(), xs.max(), 100)
  plt.plot(x_smooth,fcn(x_smooth),label='Predict')
  plt.ylabel('Shoe Size')
  plt.xlabel('Height')

  xs = np.array(xs)
  ys = np.array(ys)
  errors = ys - fcn(xs)
  clr_error = []
  for i, error in enumerate(errors):
      if error >= 0:
          clr = 'red'
      else:
          clr = 'blue'
      plt.plot([xs[i], xs[i]], [fcn(xs[i]), ys[i]], color=clr, linestyle='--')
      clr_error.append(clr)
  plt.legend()
  plt.title('Visualize error')

  plt.sca(axs[1])
  # 정사각형 넓이 리스트
  square_areas = errors*errors

  # 정사각형 색상 리스트
  colors = ['red', 'blue', 'green', 'yellow', 'purple']

  # 넓이 합 계산
  total_area = sum(square_areas)

  # 시각화 시작
  ax = axs[1]

  # 정사각형 그리기
  start_x = 0
  for i, area in enumerate(square_areas):
      side = np.sqrt(area)  # 정사각형 한 변의 길이 계산
      rect = plt.Rectangle((start_x, 0), side, side, facecolor=clr_error[i], edgecolor='k')
      ax.add_patch(rect)
      start_x += side  # 다음 정사각형 시작 위치 설정

  # 그래프 설정
  ax.set_aspect('equal')  # 정사각형 모양 유지
  plt.title('Sum of Areas of Squares')
  plt.xlabel('Width')
  plt.ylabel('Height')
  plt.plot([0, l, l, 0, 0],[0, 0, l, l, 0],'--',label=f'RMSE={l}')
  lsum = np.sqrt(square_areas.sum())
  xs_sum = np.array([0,1,1,0,0])
  ys_sum = np.array([0,0,1,1,0])
  plt.plot(xs_sum*lsum,ys_sum*lsum,label=f"RMSE(Current Model)={lsum:.3f}")
  plt.legend()

visualize_MSE(fcn_estimate,xs,ys)


## 데이터를 학습하는 인공지능

"*인공지능은 데이터로부터 스스로 학습한다.*" 라는 말은 무슨 뜻일까요? 무엇을 배우고 어떻게 배울까요? 위에서 우리는 그래프를 보고 직접 파라미터인 `a`와 `b`를 지정해 주었습니다. 사람이 데이터를 보고 직접 모델을 만들어 준 것이죠. 인공지능은 사람이 하는 일을 대신 해주죠? 인공지능, 더 정확히는 **기계학습 (machine learning)** 은 사람 대신에 데이터를 보고, 데이터를 가장 잘 설명할 수 있는 파라미터를 결정하는 과정을 **학습**이라고 부릅니다.


In [ ]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(xs.reshape(-1,1), ys.reshape(-1,1))

a = LR.coef_[0][0]
b = LR.intercept_[0]

print("a: ", a)
print("b: ", b)
print("RMSE", np.sqrt(((ys - (a * xs + b)) ** 2).sum()))